<a href="https://colab.research.google.com/github/YenOjeda/DataLab/blob/main/amazon_sale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery
!pip install pandas
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
# Especifica el ID del proyecto
project_id = 'datalab-410221'
client = bigquery.Client(project=project_id)
# Especifica la consulta SQL para obtener los datos
query = "SELECT * FROM `datalab-410221.datos.amazon_limpia`"
# Ejecuta la consulta y almacena los resultados en un DataFrame de Pandas
df = client.query(query).to_dataframe()
# Imprime el DataFrame
print(df.head())


                               review_content_limpio  sentiment_score
0  The remote works well and is pleasantly compat...           0.8750
1  Normal heat by this product  Good Quit good ht...           0.8750
2  Nice notebook with sufficiently good quality p...           0.9863
3  Good Easy to clean and use really a good one  ...           0.9863
4  good product value for money  Good product Wor...           0.3506


CÁLCULO DE SCORE DE SENTIMIENTO

In [ ]:
!pip install nltk


In [ ]:
!pip install nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autenticación en Google Cloud
auth.authenticate_user()

# Configuración del proyecto de Google Cloud
project_id = "datalab-410221"
client = bigquery.Client(project=project_id)

# Consulta en BigQuery para obtener los datos necesarios
query = """
SELECT
  review_content_limpio
FROM
  `datalab-410221.datos.amazon_limpia`
"""

# ...

# Ejecuta la consulta y obtén los resultados
query_job = client.query(query)
results = list(query_job.result())  # Convertir resultados a lista

# Inicializa una lista para almacenar los resultados del análisis de sentimientos
sentiments = []

# Analiza el sentimiento para cada revisión y almacena los resultados
sia = SentimentIntensityAnalyzer()
for row in results:
    content = row["review_content_limpio"]
    sentiment_score = sia.polarity_scores(content)["compound"]
    sentiments.append(sentiment_score)

# Crea un DataFrame con los resultados
df = pd.DataFrame({
    "review_content_limpio": [row["review_content_limpio"] for row in results],
    "sentiment_score": sentiments
})

# Imprime el DataFrame
print(df.head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


                               review_content_limpio  sentiment_score
0  The remote works well and is pleasantly compat...           0.8750
1  Normal heat by this product  Good Quit good ht...           0.8750
2  Nice notebook with sufficiently good quality p...           0.9863
3  Good Easy to clean and use really a good one  ...           0.9863
4  good product value for money  Good product Wor...           0.3506


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
# Autenticación en Google Cloud
auth.authenticate_user()
# Configuración del proyecto de Google Cloud
project_id = "datalab-410221"
client = bigquery.Client(project=project_id)
# Consulta en BigQuery para obtener los datos necesarios
query = """
SELECT
  review_content_limpio
FROM
  `datalab-410221.datos.amazon_limpia`
"""
# Ejecuta la consulta y obtén los resultados
query_job = client.query(query)
results = list(query_job.result())
# Inicializa una lista para almacenar los resultados del análisis de sentimientos
sentiments = []
# Analiza el sentimiento para cada revisión y almacena los resultados
sia = SentimentIntensityAnalyzer()
for row in results:
    content = row["review_content_limpio"]
    sentiment_score = sia.polarity_scores(content)["compound"]
    sentiments.append(sentiment_score)
# Crea un DataFrame con los resultados y agrega la nueva columna
df = pd.DataFrame({
    "review_content_limpio": [row["review_content_limpio"] for row in results],
    "sentiment_score": sentiments
})
# Crea la tabla temporal con la nueva columna
table_temporal_id = "temporal"  # Reemplaza con tu ID de tabla temporal
table_temporal_ref = client.dataset("datos").table(table_temporal_id)
job_config_temporal = bigquery.LoadJobConfig(
    schema=[bigquery.SchemaField("sentiment_score", "FLOAT64")],
    write_disposition="WRITE_TRUNCATE"
)
client.load_table_from_dataframe(df, table_temporal_ref, job_config=job_config_temporal).result()
# Recrea la vista con la nueva columna
sql_statement = f"""
CREATE OR REPLACE VIEW `datalab-410221.datos.amazon_limpia` AS
SELECT
  review_content_limpio,
  sentiment_score
FROM
  `datalab-410221.datos.{table_temporal_id}`
"""
client.query(sql_statement).result()


IMPORTAR TABLA GENERAL

In [ ]:
!pip install google-cloud-bigquery
!pip install pandas
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
# Especifica el ID del proyecto
project_id = 'datalab-410221'
client = bigquery.Client(project=project_id)
# Especifica la consulta SQL para obtener los datos
query = "SELECT * FROM `datalab-410221.datos.AmzSalesData`"
# Ejecuta la consulta y almacena los resultados en un DataFrame de Pandas
df = client.query(query).to_dataframe()
# Imprime el DataFrame
print(df.head())

   product_id                                       product_name  \
0  B078XFKBZL       Prestige Clean Home Water Purifier Cartridge   
1  B00MFPCY5C  GIZGA essentials Universal Silicone Keyboard P...   
2  B09RMQYHLH  realme narzo 50 (Speed Blue, 4GB RAM+64GB Stor...   
3  B0B9LDCX89  STRIFF Mpad Mouse Mat 230X190X3mm Gaming Mouse...   
4  B089BDBDGM  Kuber Industries Waterproof Round Laundry Bag/...   

                                            category            subcategory  \
0  Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...           Home&Kitchen   
1  Computers&Accessories|Accessories&Peripherals|...  Computers&Accessories   
2  Electronics|Mobiles&Accessories|Smartphones&Ba...            Electronics   
3  Computers&Accessories|Accessories&Peripherals|...  Computers&Accessories   
4  Home&Kitchen|HomeStorage&Organization|LaundryO...           Home&Kitchen   

   actual_price  cuartil_actual_price cat_actual_price  discounted_price  \
0           640                     1   

In [ ]:
df.describe()

,actual_price,cuartil_actual_price,discounted_price,cuartil_discounted_price,discount_percentage,cuartil_discount_percentage,rating,cuartil_rating,rating_count,cuartil_rating_count,sentiment_score,cuartil_sentiment_score
count,1211.0,1211.0,1211.0,1211.0,1211.000000,1211.0,1210.000000,1211.0,1211.0,1211.0,1209.000000,1211.0
mean,4833.161024,2.513625,2699.95128,2.503716,0.466466,2.51858,4.083140,2.486375,14569.901734,2.51858,0.903471,2.486375
std,10115.577085,1.126879,6218.846057,1.11812,0.214299,1.117972,0.304089,1.113599,33125.965297,1.117972,0.297366,1.118043
min,39.0,1.0,39.0,1.0,0.000000,1.0,2.000000,1.0,2.0,1.0,-0.978300,1.0
25%,897.0,1.5,349.0,1.5,0.320000,2.0,3.900000,1.5,1009.5,2.0,0.957300,1.0
50%,1699.0,3.0,849.0,3.0,0.490000,3.0,4.100000,2.0,4145.0,3.0,0.983400,2.0
75%,3999.0,4.0,1999.0,3.0,0.620000,4.0,4.300000,3.0,13548.0,4.0,0.993600,3.0
max,139900.0,4.0,77990.0,4.0,0.940000,4.0,5.000000,4.0,426973.0,4.0,0.999900,4.0


TESTS DE SHAPIRO: para validar distribución de los datos

In [ ]:
import pandas as pd
from scipy.stats import shapiro
from google.colab import auth

# Autenticación con BigQuery (puede que necesites autenticarte si aún no lo has hecho)
auth.authenticate_user()

# Especifica tu proyecto y configuración de BigQuery
project_id = "datalab-410221"
query = "SELECT discounted_price, rating_count FROM `datalab-410221.datos.AmzSalesData`"  # Incluye solo las columnas relevantes
# Lee los datos desde BigQuery a un DataFrame de pandas
df = pd.read_gbq(query, project_id=project_id, dialect="standard")

# Selecciona la columna de interés para aplicar el test de Shapiro para discounted_price
columna_descuento = 'discounted_price'
datos_descuento = df[columna_descuento].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para discounted_price
estadistico_shapiro_descuento, p_valor_descuento = shapiro(datos_descuento)

# Imprime los resultados para discounted_price
print(f'Estadístico de Shapiro para discounted_price: {estadistico_shapiro_descuento}')
print(f'P-valor para discounted_price: {p_valor_descuento}')

# Selecciona la columna de interés para aplicar el test de Shapiro para rating_count
columna_calificaciones = 'rating_count'
datos_calificaciones = df[columna_calificaciones].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para rating_count
estadistico_shapiro_calificaciones, p_valor_calificaciones = shapiro(datos_calificaciones)

# Imprime los resultados para rating_count
print(f'Estadístico de Shapiro para rating_count: {estadistico_shapiro_calificaciones}')
print(f'P-valor para rating_count: {p_valor_calificaciones}')

# Interpreta los resultados para discounted_price
alpha = 0.05
if p_valor_descuento > alpha:
    print('No se rechaza la hipótesis nula para discounted_price. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para discounted_price. Los datos no siguen una distribución normal.')

# Interpreta los resultados para rating_count
if p_valor_calificaciones > alpha:
    print('No se rechaza la hipótesis nula para rating_count. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para rating_count. Los datos no siguen una distribución normal.')


Estadístico de Shapiro para discounted_price: 0.40662211179733276
P-valor para discounted_price: 0.0
Estadístico de Shapiro para rating_count: 0.42857837677001953
P-valor para rating_count: 0.0
Se rechaza la hipótesis nula para discounted_price. Los datos no siguen una distribución normal.
Se rechaza la hipótesis nula para rating_count. Los datos no siguen una distribución normal.


In [ ]:
import pandas as pd
from scipy.stats import shapiro
from google.colab import auth

# Autenticación con BigQuery (puede que necesites autenticarte si aún no lo has hecho)
auth.authenticate_user()

# Especifica tu proyecto y configuración de BigQuery
project_id = "datalab-410221"
query = "SELECT actual_price, discount_percentage FROM `datalab-410221.datos.AmzSalesData`"  # Incluye solo las columnas relevantes
# Lee los datos desde BigQuery a un DataFrame de pandas
df = pd.read_gbq(query, project_id=project_id, dialect="standard")

# Selecciona la columna de interés para aplicar el test de Shapiro para actual_price
columna_descuento = 'actual_price'
datos_descuento = df[columna_descuento].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para actual_price
estadistico_shapiro_descuento, p_valor_descuento = shapiro(datos_descuento)

# Imprime los resultados para actual_price
print(f'Estadístico de Shapiro para actual_price: {estadistico_shapiro_descuento}')
print(f'P-valor para actual_price: {p_valor_descuento}')

# Selecciona la columna de interés para aplicar el test de Shapiro para discount_percentage
columna_calificaciones = 'discount_percentage'
datos_calificaciones = df[columna_calificaciones].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para discount_percentage
estadistico_shapiro_calificaciones, p_valor_calificaciones = shapiro(datos_calificaciones)

# Imprime los resultados para discount_percentage
print(f'Estadístico de Shapiro para discount_percentage: {estadistico_shapiro_calificaciones}')
print(f'P-valor para discount_percentage: {p_valor_calificaciones}')

# Interpreta los resultados para actual_price
alpha = 0.05
if p_valor_descuento > alpha:
    print('No se rechaza la hipótesis nula para actual_price. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para actual_price. Los datos no siguen una distribución normal.')

# Interpreta los resultados para discount_percentage
if p_valor_calificaciones > alpha:
    print('No se rechaza la hipótesis nula para discount_percentage. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para discount_percentage. Los datos no siguen una distribución normal.')


Estadístico de Shapiro para actual_price: 0.4349815249443054
P-valor para actual_price: 0.0
Estadístico de Shapiro para discount_percentage: 0.9792519807815552
P-valor para discount_percentage: 3.5695049346862184e-12
Se rechaza la hipótesis nula para actual_price. Los datos no siguen una distribución normal.
Se rechaza la hipótesis nula para discount_percentage. Los datos no siguen una distribución normal.


In [ ]:
import pandas as pd
from scipy.stats import shapiro
from google.colab import auth

# Autenticación con BigQuery (puede que necesites autenticarte si aún no lo has hecho)
auth.authenticate_user()

# Especifica tu proyecto y configuración de BigQuery
project_id = "datalab-410221"
query = "SELECT sentiment_score, rating FROM `datalab-410221.datos.AmzSalesData`"  # Incluye solo las columnas relevantes
# Lee los datos desde BigQuery a un DataFrame de pandas
df = pd.read_gbq(query, project_id=project_id, dialect="standard")

# Selecciona la columna de interés para aplicar el test de Shapiro para sentiment_score
columna_descuento = 'sentiment_score'
datos_descuento = df[columna_descuento].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para sentiment_score
estadistico_shapiro_descuento, p_valor_descuento = shapiro(datos_descuento)

# Imprime los resultados para sentiment_score
print(f'Estadístico de Shapiro para sentiment_score: {estadistico_shapiro_descuento}')
print(f'P-valor para sentiment_score: {p_valor_descuento}')

# Selecciona la columna de interés para aplicar el test de Shapiro para rating
columna_calificaciones = 'rating'
datos_calificaciones = df[columna_calificaciones].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para rating
estadistico_shapiro_calificaciones, p_valor_calificaciones = shapiro(datos_calificaciones)

# Imprime los resultados para rating
print(f'Estadístico de Shapiro para rating: {estadistico_shapiro_calificaciones}')
print(f'P-valor para rating: {p_valor_calificaciones}')

# Interpreta los resultados para sentiment_score
alpha = 0.05
if p_valor_descuento > alpha:
    print('No se rechaza la hipótesis nula para sentiment_score. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para sentiment_score. Los datos no siguen una distribución normal.')

# Interpreta los resultados para rating
if p_valor_calificaciones > alpha:
    print('No se rechaza la hipótesis nula para rating. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para rating. Los datos no siguen una distribución normal.')


Estadístico de Shapiro para sentiment_score: 0.3215707540512085
P-valor para sentiment_score: 0.0
Estadístico de Shapiro para rating: 0.9332689642906189
P-valor para rating: 8.415389538788812e-23
Se rechaza la hipótesis nula para sentiment_score. Los datos no siguen una distribución normal.
Se rechaza la hipótesis nula para rating. Los datos no siguen una distribución normal.


In [ ]:
import pandas as pd
from scipy.stats import shapiro
from google.colab import auth

# Autenticación con BigQuery (puede que necesites autenticarte si aún no lo has hecho)
auth.authenticate_user()

# Especifica tu proyecto y configuración de BigQuery
project_id = "datalab-410221"
query = "SELECT actual_price, sentiment_score FROM `datalab-410221.datos.AmzSalesData`"  # Incluye solo las columnas relevantes
# Lee los datos desde BigQuery a un DataFrame de pandas
df = pd.read_gbq(query, project_id=project_id, dialect="standard")

# Selecciona la columna de interés para aplicar el test de Shapiro para actual_price
columna_descuento = 'actual_price'
datos_descuento = df[columna_descuento].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para actual_price
estadistico_shapiro_descuento, p_valor_descuento = shapiro(datos_descuento)

# Imprime los resultados para actual_price
print(f'Estadístico de Shapiro para actual_price: {estadistico_shapiro_descuento}')
print(f'P-valor para actual_price: {p_valor_descuento}')

# Selecciona la columna de interés para aplicar el test de Shapiro para sentiment_score
columna_calificaciones = 'sentiment_score'
datos_calificaciones = df[columna_calificaciones].dropna()  # Asegúrate de manejar los valores nulos adecuadamente

# Aplica el test de Shapiro para sentiment_score
estadistico_shapiro_calificaciones, p_valor_calificaciones = shapiro(datos_calificaciones)

# Imprime los resultados para sentiment_score
print(f'Estadístico de Shapiro para sentiment_score: {estadistico_shapiro_calificaciones}')
print(f'P-valor para sentiment_score: {p_valor_calificaciones}')

# Interpreta los resultados para actual_price
alpha = 0.05
if p_valor_descuento > alpha:
    print('No se rechaza la hipótesis nula para actual_price. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para actual_price. Los datos no siguen una distribución normal.')

# Interpreta los resultados para sentiment_score
if p_valor_calificaciones > alpha:
    print('No se rechaza la hipótesis nula para sentiment_score. Los datos parecen seguir una distribución normal.')
else:
    print('Se rechaza la hipótesis nula para sentiment_score. Los datos no siguen una distribución normal.')

Estadístico de Shapiro para actual_price: 0.4336775541305542
P-valor para actual_price: 0.0
Estadístico de Shapiro para sentiment_score: 0.32131510972976685
P-valor para sentiment_score: 0.0
Se rechaza la hipótesis nula para actual_price. Los datos no siguen una distribución normal.
Se rechaza la hipótesis nula para sentiment_score. Los datos no siguen una distribución normal.


PRUEBA DE SIGNIFICANCIA: Test de Pearson, para el calculo de correlaciones y comprobación de hipótesis


In [ ]:
from google.cloud import bigquery
from scipy.stats import pearsonr

# Configurar el cliente de BigQuery (reemplazar 'tu-proyecto' con el nombre de tu proyecto)
client = bigquery.Client(project='datalab-410221')

# Especificar la consulta (reemplazar 'tu-dataset.tu-tabla' con tus propios datos)
query = """
SELECT discounted_price, rating
FROM `datalab-410221.datos.AmzSalesData`
"""

# Ejecutar la consulta y obtener los resultados en un DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()

# Eliminar filas con valores faltantes
df = df.dropna()

# Calcular la correlación de Pearson y realizar la prueba de significancia
r, p_value = pearsonr(df['rating'], df['discounted_price'])

# Imprimir los resultados
print(f"Correlación de Pearson: {r}")
print(f"Valor p: {p_value}")

# Comparar con un umbral de significancia (por ejemplo, 0.05)
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula. Hay una correlación significativa.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")


Correlación de Pearson: 0.1208351423137678
Valor p: 2.5242778219956965e-05
Se rechaza la hipótesis nula. Hay una correlación significativa.


In [ ]:
from google.cloud import bigquery
from scipy.stats import pearsonr
# Configurar el cliente de BigQuery (reemplazar 'tu-proyecto' con el nombre de tu proyecto)
client = bigquery.Client(project='datalab-410221')
# Especificar la consulta (reemplazar 'tu-dataset.tu-tabla' con tus propios datos)
query = """
SELECT actual_price, discount_percentage
FROM `datalab-410221.datos.AmzSalesData`
"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()
# Calcular la correlación de Pearson y realizar la prueba de significancia
r, p_value = pearsonr(df['discount_percentage'], df['actual_price'])
# Imprimir los resultados
print(f"Correlación de Pearson: {r}")
print(f"Valor p: {p_value}")
# Comparar con un umbral de significancia (por ejemplo, 0.05)
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula. Hay una correlación significativa.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")

Correlación de Pearson: -0.05770229410746708
Valor p: 0.044685191062970635
Se rechaza la hipótesis nula. Hay una correlación significativa.


In [ ]:
from google.cloud import bigquery
from scipy.stats import pearsonr

# Configurar el cliente de BigQuery (reemplazar 'tu-proyecto' con el nombre de tu proyecto)
client = bigquery.Client(project='datalab-410221')

# Especificar la consulta (reemplazar 'tu-dataset.tu-tabla' con tus propios datos)
query = """
SELECT sentiment_score, rating
FROM `datalab-410221.datos.AmzSalesData`
"""

# Ejecutar la consulta y obtener los resultados en un DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()

# Eliminar filas con valores faltantes
df = df.dropna()

# Calcular la correlación de Pearson y realizar la prueba de significancia
r, p_value = pearsonr(df['rating'], df['sentiment_score'])

# Imprimir los resultados
print(f"Correlación de Pearson: {r}")
print(f"Valor p: {p_value}")

# Comparar con un umbral de significancia (por ejemplo, 0.05)
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula. Hay una correlación significativa.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")


Correlación de Pearson: 0.22667087360903232
Valor p: 1.576657854390637e-15
Se rechaza la hipótesis nula. Hay una correlación significativa.


In [ ]:
from google.cloud import bigquery
from scipy.stats import pearsonr

# Configurar el cliente de BigQuery (reemplazar 'tu-proyecto' con el nombre de tu proyecto)
client = bigquery.Client(project='datalab-410221')

# Especificar la consulta (reemplazar 'tu-dataset.tu-tabla' con tus propios datos)
query = """
SELECT sentiment_score, actual_price
FROM `datalab-410221.datos.AmzSalesData`
"""

# Ejecutar la consulta y obtener los resultados en un DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()

# Eliminar filas con valores faltantes
df = df.dropna()

# Calcular la correlación de Pearson y realizar la prueba de significancia
r, p_value = pearsonr(df['actual_price'], df['sentiment_score'])

# Imprimir los resultados
print(f"Correlación de Pearson: {r}")
print(f"Valor p: {p_value}")

# Comparar con un umbral de significancia (por ejemplo, 0.05)
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula. Hay una correlación significativa.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")

Correlación de Pearson: 0.03410254015741479
Valor p: 0.23586824846351448
No hay suficiente evidencia para rechazar la hipótesis nula.
